今回扱うガウスの超幾何関数$F(a,b,c,z)$は次のセルの式で定義されます。ここで登場する$(a)_n$はポッホハマー記号と呼ばれ次の式で定義されます。
$$(a)_n=\prod_{k=0}^{n-1}(a+k)=\frac{\Gamma(a+n)}{\Gamma(a)}$$

In [1]:
hypergeometric([a,b],[c],z)=sum(pochhammer(a,n)*pochhammer(b,n)/(pochhammer(c,n)*n!)*z^n,n,0,inf);

                                               inf              n
                                               ====  (a)  (b)  z
                                               \        n    n
(%o1)         hypergeometric([a, b], [c], z) =  >    ------------
                                               /       (c)  n!
                                               ====       n
                                               n = 0

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining MAXIMA::SIMP-HYPERGEOMETRIC in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining MAXIMA::SIMP-UNIT-STEP in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining MAXIMA::SIMP-POCHHAMMER in DEFUN


超幾何関数は$a,b,c,z$に様々な値や式を代入することで、多くの初等関数や特殊関数を表すことができます。ここでは$\cos{z}$が次の超幾何関数で表されることを見てます。確認のために次の超幾何関数と$\cos{z}$をテイラー展開して係数が一致することを見てみましょう。

In [2]:
hypergeometric([],[1/2],-z^2/4);

                                                    2
                                             1     z
(%o2)                    hypergeometric([], [-], - --)
                                             2     4

In [3]:
taylor(%o2,z,0,15);

               14            12         10       8      6     4    2
              z             z          z        z      z     z    z
(%o3)  (- -----------) + --------- - ------- + ----- - --- + -- - -- + 1
          87178291200    479001600   3628800   40320   720   24   2

In [4]:
taylor(cos(z),z,0,15);

               14            12         10       8      6     4    2
              z             z          z        z      z     z    z
(%o4)  (- -----------) + --------- - ------- + ----- - --- + -- - -- + 1
          87178291200    479001600   3628800   40320   720   24   2

超幾何関数が初等関数や特殊関数で表せる場合に、Maximaではそのような簡約を実行することができます。%o2として定義した超幾何関数を簡約してみましょう。

In [5]:
besselexpand:true$
hypergeometric_simp(%o2);

(%o6)                               cos(z)

In [6]:
hypergeometric([],[1/2],-z^2/4);

                                                    2
                                             1     z
(%o7)                    hypergeometric([], [-], - --)
                                             2     4

上記の式で定義された超幾何関数が$\cos{z}$に等しいことを級数展開の係数が一致することで証明します。
$$\cos{z}=\sum_{n=0}^{\infty}\frac{(-1)^n\,z^{2\,n}}{(2\,n)!}$$
は既知とします。
超幾何関数の定義式より、$F\left( \left. \begin{array}{c}\\c\end{array} \right |,x\right)=\sum_{n=0}^{\infty}\frac{x^n}{(c)_n\,n!}$ですから、この総和記号の中身と$\cos{z}$の展開式の総和記号の中身が一致することを示します。

In [7]:
declare(n,integer)$
1/pochhammer(c,n)*x^n/n!;

                                       n
                                      x
(%o9)                               -------
                                    (c)  n!
                                       n

ポッホハマー記号および階乗記号をガンマ関数に書き直してみます。

In [8]:
makegamma(%);

                                            n
                                  gamma(c) x
(%o10)                     -------------------------
                           gamma(n + 1) gamma(n + c)

ここに$c=\frac{1}{2}, x=-\frac{z^2}{4}$を代入してみます。

In [9]:
CTERM:%,c=1/2,x=-z^2/4;

                                           n  2 n
                            sqrt(%pi) (- 1)  z
(%o11)                   ----------------------------
                          n           1
                         4  gamma(n + -) gamma(n + 1)
                                      2

半整数に対するガンマ関数の計算をルールとして定義してそれを適用します。定義しているルールは、
$$\Gamma(n+\frac{1}{2})=\sqrt{\pi}\,\frac{(2\,n-1)!!}{2^n}$$
です。ただ、Maximaの2重階乗の処理の都合でgenfact関数に変換されて表示されます。

In [10]:
matchdeclare(exp,lambda([exp],featurep(exp,integer)))$
defrule(gamma_half_int,gamma(exp+1/2),sqrt(%pi)*(2*exp-1)!!/2^exp);

                                    1
(%o13) gamma_half_int : gamma(exp + -) -> 
                                    2
                                                                  2 exp - 1
                                     sqrt(%pi) genfact(2 exp - 1, ---------, 2)
                                                                      2
                                     ------------------------------------------
                                                         exp
                                                        2

In [11]:
apply1(CTERM,gamma_half_int);

                                     n  n  2 n
                                (- 1)  2  z
(%o14)           --------------------------------------------
                  n                               2 n - 1
                 4  gamma(n + 1) genfact(2 n - 1, -------, 2)
                                                     2

In [12]:
%,radcan;

                                      n  2 n
                                 (- 1)  z
(%o15)           --------------------------------------------
                  n                               2 n - 1
                 2  gamma(n + 1) genfact(2 n - 1, -------, 2)
                                                     2

色々と約分されてようやくここまで変形できました。$genfact(2\,n-1,\frac{(2\,n-1)}{2},2)=(2\,n-1)!!$であることを思い出してください。これは$2\,n-1$以下の全ての奇数の積です。

$\Gamma(n+1)=n!$なのでこれは$n$以下の全ての自然数の積です。最初の$2^n$をこの$n!$に分けてかけると、$2\,n$以下の全ての偶数の積が得られます。つまり$2^n\cdot \Gamma(n+1)=(2\,n)!!$です。

したがって上記の式の分母は$(2\,n)!!\cdot(2\,n-1)!!$となるのですが、この積は結局$2\,n$以下の全ての自然数の積になります。つまり$(2\,n)!$に等しくなります。というわけで上記の式は次の式に等しくなります。
$$\frac{\left(-1\right)^{n}\,z^{2\,n}}{(2\,n)!}$$

$\cos{z}$のテイラー展開と一致していることが分かります。
（証明終了）

## 超幾何関数の定義とcos zの超幾何関数表現

Copyright 2021 Yasuaki Honda (yasuaki.honda@gmail.com)
GPL2

If you want to use this file with different licensing terms, please email me.